In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#---CODE MODIFICATION: NEW LIBRERIES TO BE IMPORTED
import os

In [ ]:
#clone the git repo that contains the data and additional information about the dataset
#---if the repo is already cloned, skip this step
if not os.path.exists('WANDS'):
    # Only clone if directory does not exist
    !git clone https://github.com/wayfair/WANDS.git
else:
    print("WANDS repo already cloned. Skipping git clone step.")


WANDS repo already cloned. Skipping git clone step.


# TF-IDF and information theory

### 🔍 Information-Theoretic View of TF–IDF (Motivation)

In this section we briefly analyze the original TF–IDF technique from an information-theoretic perspective, as motivation for the methods we will use in this assignment.

---

### 1. Vector Representation of Product Documents and Queries

Let $V$ be the vocabulary of all unique terms (words) in the corpus, with size $|V|$.

- Each **product document** $d$ is represented as a vector
  $$
  \mathbf{v}^{(d)} = (w_{t_1,d}, w_{t_2,d}, \dots, w_{t_{|V|},d}) \in \mathbb{R}^{|V|}
  $$
- Each **query** $q$ is represented in the same space:
  $$
  \mathbf{v}^{(q)} = (w_{t_1,q}, w_{t_2,q}, \dots, w_{t_{|V|},q}) \in \mathbb{R}^{|V|}
  $$

Here, $w_{t,d}$ is the TF–IDF weight of term $t$ in product document $d$.  
The goal is to embed both product documents and queries in a common vector space so that we can compute similarity (e.g., cosine similarity) to rank products.

---

### 2. Term Frequency (TF) as Local Probability

Let $t$ be a term and $d$ a product document. A normalized term frequency is:

$$
\mathrm{TF}(t,d) = \frac{f_{t,d}}{\sum_j f_{j,d}}
$$

where:

- $f_{t,d}$ is the number of occurrences of term $t$ in product document $d$,
- $\sum_j f_{j,d}$ is the total number of term occurrences in $d$.

From a probabilistic viewpoint, this can be interpreted as an **empirical estimate of the local probability** of seeing $t$ in product document $d$:

$$
\mathrm{TF}(t,d) \approx \hat{p}(t \mid d)
$$

So TF captures how important $t$ is **within this particular product document** $d$.  
It is inherently **product-document–dependent**.

---

### 3. Inverse Document Frequency (IDF) as Global Surprise

Now consider the corpus as a whole:

- $N$ = total number of product documents.
- $df(t)$ = number of product documents that contain term $t$.

The inverse document frequency is defined as:

$$
\mathrm{IDF}(t) = \log\left(\frac{N}{1 + df(t)}\right)
$$

If we approximate the **global probability** that a random product document contains $t$ by

$$
\hat{p}(t) \approx \frac{df(t)}{N},
$$

then:

$$
\mathrm{IDF}(t) \approx -\log \hat{p}(t)
$$

This is the **self-information** or **surprisal** of observing term $t$ at the corpus level.

- Common terms (high $\hat{p}(t)$) get low IDF.
- Rare terms (low $\hat{p}(t)$) get high IDF.

Importantly, **IDF depends only on the term** $t$, not on any specific product document $d$.

---

### 4. TF–IDF: Local Probability × Global Surprise

The TF–IDF weight for term $t$ in product document $d$ is:

$$
w_{t,d} = \mathrm{TF\text{-}IDF}(t,d) = \mathrm{TF}(t,d)\,\mathrm{IDF}(t).
$$

Using the information-theoretic interpretations:

- $\mathrm{TF}(t,d) \approx \hat{p}(t \mid d)$ (local term probability in product document $d$),
- $\mathrm{IDF}(t) \approx -\log \hat{p}(t)$ (global surprisal of $t$),

we get:

$$
w_{t,d}
\;\approx\;
\hat{p}(t \mid d)\,\big[-\log \hat{p}(t)\big].
$$

So, for a fixed product document $d$, its TF–IDF vector is:

$$
\mathbf{v}^{(d)} =
\Big(
\hat{p}(t_1 \mid d)\,[-\log \hat{p}(t_1)],
\;
\hat{p}(t_2 \mid d)\,[-\log \hat{p}(t_2)],
\;
\dots,
\;
\hat{p}(t_{|V|} \mid d)\,[-\log \hat{p}(t_{|V|})]
\Big).
$$

This explains why **each vector component depends on the product document $d$**:

- The **local factor** $\hat{p}(t \mid d)$ changes from product document to product document.
- The **global factor** $-\log \hat{p}(t)$ is shared across the corpus.

Information-theoretic intuition:

- $-\log \hat{p}(t)$ is the information content of seeing term $t$ in general.
- $\hat{p}(t \mid d)$ scales that information by how characteristic $t$ is of product document $d$.

---
### 5. Vocabulary and Query Representation in Practice

In practice, the TF–IDF **vocabulary** is built **only from product documents**, not from user queries:

- We collect all product documents (titles, descriptions, etc.).
- We apply tokenization and text preprocessing.
- We **fit** the TF–IDF vectorizer on these product documents:
  - this fixes the vocabulary $V = \{t_1, \dots, t_{|V|}\}$,
  - computes $df(t)$ for each term,
  - and derives the corresponding $\mathrm{IDF}(t)$.

A user **query** $q$ is then treated as a short text and projected into this **same vocabulary space**:

- We apply the same preprocessing to $q$.
- We compute TF for query terms (often with a simpler normalization).
- We reuse the existing $\mathrm{IDF}(t)$ values learned from product documents.

Analogously to the product-document case, we can write:

- Query term frequency (local, in the query):
  $$
  \mathrm{TF}(t, q) = \frac{f_{t,q}}{\sum_j f_{j,q}}
  \;\approx\;
  \hat{p}(t \mid q)
  $$
- Global IDF (reused from product documents):
  $$
  \mathrm{IDF}(t) \approx -\log \hat{p}(t)
  $$

So each component of the **query vector** is:

$$
w_{t,q} = \mathrm{TF\text{-}IDF}(t,q)
= \mathrm{TF}(t,q)\,\mathrm{IDF}(t)
\;\approx\;
\hat{p}(t \mid q)\,\big[-\log \hat{p}(t)\big].
$$

Explicitly, the query vector in the shared TF–IDF space is:

$$
\mathbf{v}^{(q)} =
\Big(
\hat{p}(t_1 \mid q)\,[-\log \hat{p}(t_1)],
\;
\hat{p}(t_2 \mid q)\,[-\log \hat{p}(t_2)],
\;
\dots,
\;
\hat{p}(t_{|V|} \mid q)\,[-\log \hat{p}(t_{|V|})]
\Big)
\in \mathbb{R}^{|V|}.
$$

Important practical detail:

- If a query term $t_k$ **exists** in the product vocabulary $V$, it receives a TF–IDF weight $w_{t_k,q}$ as above.
- If $t_k$ **never appears** in any product document, it is **out-of-vocabulary (OOV)** and is simply ignored (no coordinate in the vector).

This ensures that:

- Both product documents and queries live in the **same TF–IDF vector space**, defined by the product catalog.
- All global statistics (like $\hat{p}(t)$ and $\mathrm{IDF}(t)$) are meaningful with respect to the product documents we are actually ranking.

---
### 6. Product-Document Matrix View (Explicit Rectangular Form)

For each product document $d_i$ and term $t_j \in V$, the TF–IDF weight can be written (using the information-theoretic approximations) as:

$$
\mathrm{TF\text{-}IDF}(t_j, d_i)
\;\approx\;
\hat{p}(t_j \mid d_i)\,\big[-\log \hat{p}(t_j)\big],
$$

where:

- $\hat{p}(t_j \mid d_i)$ is the empirical **local probability** of term $t_j$ in product document $d_i$ (from TF),
- $-\log \hat{p}(t_j)$ is the **global surprisal** of term $t_j$ in the corpus (from IDF).

If we stack all $M$ product documents, the **product-document TF–IDF matrix** can be written explicitly as:

$$
\mathbf{X}
\;\approx\;
\begin{bmatrix}
\hat{p}(t_1 \mid d_1)\,[-\log \hat{p}(t_1)] & \hat{p}(t_2 \mid d_1)\,[-\log \hat{p}(t_2)] & \dots & \hat{p}(t_{|V|} \mid d_1)\,[-\log \hat{p}(t_{|V|})] \\
\hat{p}(t_1 \mid d_2)\,[-\log \hat{p}(t_1)] & \hat{p}(t_2 \mid d_2)\,[-\log \hat{p}(t_2)] & \dots & \hat{p}(t_{|V|} \mid d_2)\,[-\log \hat{p}(t_{|V|})] \\
\vdots & \vdots & \ddots & \vdots \\
\hat{p}(t_1 \mid d_M)\,[-\log \hat{p}(t_1)] & \hat{p}(t_2 \mid d_M)\,[-\log \hat{p}(t_2)] & \dots & \hat{p}(t_{|V|} \mid d_M)\,[-\log \hat{p}(t_{|V|})]
\end{bmatrix}
\in \mathbb{R}^{M \times |V|}.
$$

- Row $i$ is the explicit TF–IDF vector for product document $d_i$:
  $$
  \mathbf{v}^{(d_i)} =
  \Big(
  \hat{p}(t_1 \mid d_i)\,[-\log \hat{p}(t_1)],
  \dots,
  \hat{p}(t_{|V|} \mid d_i)\,[-\log \hat{p}(t_{|V|})]
  \Big).
  $$
- Column $j$ corresponds to term $t_j$ and shows how its information-weighted presence varies across product documents.

The query vector $\mathbf{v}^{(q)}$ from Section 5 lies in the **same** $|V|$-dimensional space, so retrieval becomes a matter of comparing $\mathbf{v}^{(q)}$ with the rows of $\mathbf{X}$ (e.g., via cosine similarity) to rank product documents by how similar their **information profile** is to the query.

---

### 7. Summing the entire matrix $X$: $H(T)$ emerges (and why that matters)

Let $X \in \mathbb{R}^{M \times |V|}$ with entries
$$
X_{i,j} = \hat{p}(t_j \mid d_i)\,[-\log \hat{p}(t_j)].
$$

Without assuming that documents are equiprobable, fix any prior $\pi(d_i) \ge 0$ with $\sum_{i=1}^{M} \pi(d_i) = 1$, and define the **global term profile** as the mixture
$$
\hat{p}(t_j) = \sum_{i=1}^{M} \pi(d_i)\,\hat{p}(t_j \mid d_i).
$$

Then the **prior-weighted average** of all entries of $X$ is
$$
\sum_{i=1}^{M} \pi(d_i) \sum_{j=1}^{|V|} X_{i,j}
= \sum_{j=1}^{|V|} \left( \sum_{i=1}^{M} \pi(d_i)\,\hat{p}(t_j \mid d_i) \right)[-\log \hat{p}(t_j)]
= \sum_{j=1}^{|V|} \hat{p}(t_j)\,[-\log \hat{p}(t_j)]
= H(T).
$$

Here, $T$ is the **global term variable** obtained by:

1. picking a document $d_i$ according to $\pi$, and  
2. picking a token uniformly at random within that document.

Under this sampling scheme, the entropy of $T$ is
$$
H(T) = -\sum_{j} \hat{p}(t_j)\,\log \hat{p}(t_j),
$$
which we can interpret as the **catalog entropy**: the average uncertainty about which word you see when you look at a random token in the product catalog.

A useful special case is when $\pi$ is **uniform** over documents. Then $\pi(d_i) = 1/M$, and
$$
\frac{1}{M} \sum_{i,j} X_{i,j} = H(T),
$$
that is,
$$
\sum_{i,j} X_{i,j} = M \cdot H(T).
$$

This immediately shows that TF–IDF already encodes information theory:

- Each entry is *local probability × global surprisal*:  
  $\hat{p}(t_j \mid d_i)$ captures how characteristic the term $t_j$ is within product document $d_i$, while $-\log \hat{p}(t_j)$ measures how informative that term is across the whole catalog.
- Aggregating entries yields canonical information-theoretic quantities:
  - the prior-weighted global average gives the entropy $H(T)$;
  - for a fixed document $d_i$, the sum
    $$
    \sum_{j} X_{i,j}
    $$
    is a **cross-entropy** against the global background distribution:
    $$
    \sum_{j} X_{i,j}
    = H\big(\hat{p}(\cdot \mid d_i)\big)
      + D_{\mathrm{KL}}\big(\hat{p}(\cdot \mid d_i) \,\|\, \hat{p}(\cdot)\big).
    $$

In short, TF–IDF balances **local distinctiveness** with **global informativeness**—no extra assumptions required.

#### How we will use this perspective going forward

1. **Product representations.**  
   We will use information-theoretic intuition to reason about how we embed product documents into a latent space (TF–IDF, learned embeddings, or hybrids). Dimension, scaling, and normalization become decisions about how to distribute information across coordinates.

2. **Query handling.**  
   Once a good product space is set, we will:
   - analyze how user queries are projected into the same space,
   - measure how much information queries carry relative to the catalog,
   - and adjust query-side processing and scoring to improve retrieval behavior.

In short, whether we keep TF–IDF, move to semantic embeddings, or add LLM-based retrieval, we will start from this information-theoretic angle to:

- structure the vector spaces,
- reason about what each dimension “means” in terms of information,
- and design improvements on both product representations and query handling to upgrade the baseline search engine.


# Data analisys

First, we need to analyze the data, particularly the product data, because the actual queries might differ from the query dataset, but the potential products and their characteristics are static in this case. This allows us to extract descriptive information about the necessary latent space, such as its minimum dimensions, using an information theory approach.

This is why, if we choose to represent products and queries in the same n-dimensional space, as is the case with TF-IDF, where the dimensional length is the number of words in the vocabulary, the most reasonable approach is to define this space based on the product data and, once defined, proceed to adjust the rest. In this way, we will analyze how much we can compress the product information, which features we will use, and how we will represent them to avoid distortion. Once we know the optimal representation for the products, we can define this as our latent space and implement the necessary changes to the queries to embed them in that space and perform our comparison. The idea will be to find a minimal dimension space, properly designed to maintain the semantic properties of the texts once embedded.

In [20]:
# get products
product_df = pd.read_csv("WANDS/dataset/product.csv", sep='\t')
product_df.head()

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


# Continue the assignment

In [3]:
#define functions for product search using Tf-IDF
def calculate_tfidf(dataframe):
    """
    Calculate the TF-IDF for combined product name and description.

    Parameters:
    dataframe (pd.DataFrame): DataFrame with product_id, and other product information.

    Returns:
    TfidfVectorizer, csr_matrix: TF-IDF vectorizer and TF-IDF matrix.
    """
    # Combine product name and description to vectorize
    # NOTE: Please feel free to use any combination of columns available, some columns may contain NULL values
    combined_text = dataframe['product_name'] + ' ' + dataframe['product_description']
    vectorizer = TfidfVectorizer()
    # convert combined_text to list of unicode strings
    tfidf_matrix = vectorizer.fit_transform(combined_text.values.astype('U'))
    return vectorizer, tfidf_matrix

def get_top_products(vectorizer, tfidf_matrix, query, top_n=10):
    """
    Get top N products for a given query based on TF-IDF similarity.

    Parameters:
    vectorizer (TfidfVectorizer): Trained TF-IDF vectorizer.
    tfidf_matrix (csr_matrix): TF-IDF matrix for the products.
    query (str): Search query.
    top_n (int): Number of top products to return.

    Returns:
    list: List of top N product IDs.
    """
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_product_indices = cosine_similarities.argsort()[-top_n:][::-1]
    return top_product_indices

In [4]:
#define functions for evaluating retrieval performance
def map_at_k(true_ids, predicted_ids, k=10):
    """
    Calculate the Mean Average Precision at K (MAP@K).

    Parameters:
    true_ids (list): List of relevant product IDs.
    predicted_ids (list): List of predicted product IDs.
    k (int): Number of top elements to consider.
             NOTE: IF you wish to change top k, please provide a justification for choosing the new value

    Returns:
    float: MAP@K score.
    """
    #if either list is empty, return 0
    if not len(true_ids) or not len(predicted_ids):
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids and p_id not in predicted_ids[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(true_ids), k)

In [5]:
# Please add any new evaluation functions here

In [ ]:
# get search queries
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')
query_df.head()

In [10]:
# get manually labeled groundtruth lables
label_df = pd.read_csv("WANDS/dataset/label.csv", sep='\t')

In [11]:
label_df.head()

,id,query_id,product_id,label
0,0,0,25434,Exact
1,1,0,12088,Irrelevant
2,2,0,42931,Exact
3,3,0,2636,Exact
4,4,0,42923,Exact


In [12]:
#group the labels for each query to use when identifying exact matches
grouped_label_df = label_df.groupby('query_id')

In [13]:
# Calculate TF-IDF
vectorizer, tfidf_matrix = calculate_tfidf(product_df)

In [14]:
#Sanity check code block to see if the search results are relevant
#implementing a function to retrieve top K product IDs for a query
def get_top_product_ids_for_query(query):
    top_product_indices = get_top_products(vectorizer, tfidf_matrix, query, top_n=10)
    top_product_ids = product_df.iloc[top_product_indices]['product_id'].tolist()
    return top_product_ids

#define the test query
query = "armchair"

#obtain top product IDs
top_product_ids = get_top_product_ids_for_query(query)

print(f"Top products for '{query}':")
for product_id in top_product_ids:
    product = product_df.loc[product_df['product_id'] == product_id]
    print(product_id, product['product_name'].values[0])

Top products for 'armchair':
12756 24.41 '' wide tufted polyester armchair
42698 donham armchair
42697 donham 25 '' wide armchair
41270 almaraz 33.7 '' wide leather match armchair
23907 faizah 27.6 '' wide tufted polyester armchair
31564 biloxi 34.75 '' wide armchair
41306 hartsell 33 '' wide armchair
1527 howington 39 '' wide tufted linen armchair
42802 donham polyester lounge chair
6532 ogan 29 '' wide polyester armchair


In [15]:
#implementing a function to retrieve exact match product IDs for a query_id
def get_exact_matches_for_query(query_id):
    query_group = grouped_label_df.get_group(query_id)
    exact_matches = query_group.loc[query_group['label'] == 'Exact']['product_id'].values
    return exact_matches

#applying the function to obtain top product IDs and adding top K product IDs to the dataframe 
query_df['top_product_ids'] = query_df['query'].apply(get_top_product_ids_for_query)

#adding the list of exact match product_IDs from labels_df
query_df['relevant_ids'] = query_df['query_id'].apply(get_exact_matches_for_query)

#now assign the map@k score
query_df['map@k'] = query_df.apply(lambda x: map_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1)


In [16]:
# calculate the MAP across the entire query set
query_df.loc[:, 'map@k'].mean()

0.29319550540123457